Code to segment and cluster Toronto neighborhoods based on data scraped from the web

In [36]:
import pandas as pd
import numpy as np
import requests
#import json
#from pandas.io.json import json_normalize

In [37]:
#set wikipedia url that shows Toronto neighborhoods and postal codes
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [38]:
df_raw = pd.read_html(wiki_url)

In [39]:
#Read the first table on the page into a data frame (assuming that the first table contains the neighborhood data)
neighborhoods = pd.DataFrame(df_raw[0])

In [40]:
#drop unassigned post codes
neighborhoods=neighborhoods[neighborhoods['Borough']!='Not assigned'].reset_index(drop=True)

In [41]:
neighborhoods.shape

(103, 3)